<H2>Постановка задачи</H2>
<ul>
    <li>Есть данные компании, которая разрабатывает мобильные игры. Необходимо написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).</li>
</ul>


<H4>Сведения о таблицах</H4>
<ul>
    <li>reg_data.csv – данные о времени регистрации</li>
    <li>auth_data.csv – данные о времени захода пользователей в игру</li>
</ul>

In [1]:
#загружаем библиотеки
import pandas as pd
from datetime import timedelta

In [2]:
#загружаем таблицы в переменные
df_auth_data = pd.read_csv('auth_data.csv', sep=';')
df_reg_data = pd.read_csv('reg_data.csv', sep=';')

In [3]:
#Функция для проверки значений таблицы
def check_df_values(df, df_name):
    df_info = pd.DataFrame(columns= ['',
                                     'Записей',
                                     'Уникальных',
                                     'Дублей',
                                     'NaN',
                                     'не NaN',
                                     'Min',
                                     'Max',
                                     'Тип'],
                              data=[[f'Сведения о "{df_name}"',
                                     df.shape[0],
                                     df.nunique().sum(),
                                     df.duplicated().sum(),
                                     df.isna().sum().sum(),
                                     df.notna().sum().sum(),
                                     '',
                                     '',
                                     '']])
    for col in df.columns:
        df_info = pd.concat([df_info, 
              pd.DataFrame(columns= ['',
                                     'Записей',
                                     'Уникальных',
                                     'Дублей',
                                     'NaN',
                                     'не NaN',
                                     'Min',
                                     'Max',
                                     'Тип'],
                              data=[[f'Колонка "{col}"', 
                                     df[col].shape[0], 
                                     df[col].nunique(),
                                     df[col].duplicated().sum(),
                                     df[col].isna().sum(),
                                     df[col].notna().sum(),
                                     df[col].min(),
                                     df[col].max(),
                                     df[col].dtype]])],
                              ignore_index=True)
    return df_info

In [4]:
check_df_values(df_auth_data, 'df_auth_data')

,,Записей,Уникальных,Дублей,NaN,не NaN,Min,Max,Тип
0,"Сведения о ""df_auth_data""",9601013,10180915,0,0,19202026,,,
1,"Колонка ""auth_ts""",9601013,9180915,420098,0,9601013,911382223,1600874244,int64
2,"Колонка ""uid""",9601013,1000000,8601013,0,9601013,1,1110622,int64


In [5]:
check_df_values(df_reg_data, 'df_reg_data')

,,Записей,Уникальных,Дублей,NaN,не NaN,Min,Max,Тип
0,"Сведения о ""df_reg_data""",1000000,2000000,0,0,2000000,,,
1,"Колонка ""reg_ts""",1000000,1000000,0,0,1000000,911382223,1600874244,int64
2,"Колонка ""uid""",1000000,1000000,0,0,1000000,1,1110622,int64


In [6]:
df_auth_data

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2
...,...,...
9601008,1600874034,1110618
9601009,1600874086,1110619
9601010,1600874139,1110620
9601011,1600874191,1110621


In [7]:
df_reg_data

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5
...,...,...
999995,1600874034,1110618
999996,1600874086,1110619
999997,1600874139,1110620
999998,1600874191,1110621


<li>Колонки auth_ts и reg_ts должны иметь тип даты. Поэтому преобразуем колонки в нужный тип данных. А также избавимся от значений времени (часы, минуты, секунды), чтобы мы могли строить когорту по дням.

In [8]:
#Замечание по поводу применения pd.to_datetime(pd.to_datetime(...))
#вложенный метод "to_datetime" приводит к типу object, 
#а второй метод "to_datetime" уже приводит к типу "datetime64"
df_auth_data['auth_ts'] = pd.to_datetime(pd.to_datetime(df_auth_data['auth_ts'], unit='s').dt.date)
df_reg_data['reg_ts'] = pd.to_datetime(pd.to_datetime(df_reg_data['reg_ts'], unit='s').dt.date)

In [9]:
check_df_values(df_auth_data, 'df_auth_data')

,,Записей,Уникальных,Дублей,NaN,не NaN,Min,Max,Тип
0,"Сведения о ""df_auth_data""",9601013,1006169,0,0,19202026,,,
1,"Колонка ""auth_ts""",9601013,6169,9594844,0,9601013,1998-11-18 00:00:00,2020-09-23 00:00:00,datetime64[ns]
2,"Колонка ""uid""",9601013,1000000,8601013,0,9601013,1,1110622,int64


In [10]:
check_df_values(df_reg_data, 'df_reg_data')

,,Записей,Уникальных,Дублей,NaN,не NaN,Min,Max,Тип
0,"Сведения о ""df_reg_data""",1000000,1005110,0,0,2000000,,,
1,"Колонка ""reg_ts""",1000000,5110,994890,0,1000000,1998-11-18 00:00:00,2020-09-23 00:00:00,datetime64[ns]
2,"Колонка ""uid""",1000000,1000000,0,0,1000000,1,1110622,int64


<H4>Анализ данных после преобразования</H4>
<ul>
    <li>Убрав время у колонок auth_ts и reg_ts видим, что таблицы не имеют задублированных строк. Следовательно, каждая строка таблиц - это уникальная комбинация значений даты и ИД пользователя.</li>
    <li>Таблицы не имеют пропусков в данных (NaN = 0).</li>
    <li>В таблице "auth_data" 6169 уникальных дат с 1998 по 2020 годы.</li>
    <li>В таблице "reg_data" 5110 уникальных дат с 1998 по 2020 годы.</li>
</ul>

<H4>Объединим таблицы вместе</H4>

In [11]:
df_merged = df_reg_data.merge(df_auth_data, on='uid', how='left')

In [12]:
check_df_values(df_merged, 'df_merged')

,,Записей,Уникальных,Дублей,NaN,не NaN,Min,Max,Тип
0,"Сведения о ""df_merged""",9601013,1011279,0,0,28803039,,,
1,"Колонка ""reg_ts""",9601013,5110,9595903,0,9601013,1998-11-18 00:00:00,2020-09-23 00:00:00,datetime64[ns]
2,"Колонка ""uid""",9601013,1000000,8601013,0,9601013,1,1110622,int64
3,"Колонка ""auth_ts""",9601013,6169,9594844,0,9601013,1998-11-18 00:00:00,2020-09-23 00:00:00,datetime64[ns]


In [13]:
df_merged.head()

,reg_ts,uid,auth_ts
0,1998-11-18,1,1998-11-18
1,1999-07-22,2,1999-07-22
2,1999-07-22,2,1999-07-25
3,1999-07-22,2,1999-07-31
4,1999-07-22,2,1999-08-05


In [14]:
#создадим колонку с разницей в днях между датой регистрации и датой авторизации
df_merged['days_diff'] = (df_merged["auth_ts"] - df_merged["reg_ts"]).dt.days

In [15]:
df_merged

,reg_ts,uid,auth_ts,days_diff
0,1998-11-18,1,1998-11-18,0
1,1999-07-22,2,1999-07-22,0
2,1999-07-22,2,1999-07-25,3
3,1999-07-22,2,1999-07-31,9
4,1999-07-22,2,1999-08-05,14
...,...,...,...,...
9601008,2020-09-23,1110618,2020-09-23,0
9601009,2020-09-23,1110619,2020-09-23,0
9601010,2020-09-23,1110620,2020-09-23,0
9601011,2020-09-23,1110621,2020-09-23,0


In [16]:
#сгруппируем ДФ по дате регистрации и разнице дней, чтобы сосчитать количество пользователей
cohort_df = df_merged.groupby(['reg_ts', 'days_diff'], as_index=False)\
                    .agg({'uid' : 'nunique'})\
                    .rename(columns={'uid' : 'uid_count', 'reg_ts' : 'reg_date'})
cohort_df

,reg_date,days_diff,uid_count
0,1998-11-18,0,1
1,1999-07-22,0,1
2,1999-07-22,3,1
3,1999-07-22,9,1
4,1999-07-22,14,1
...,...,...,...
2716223,2020-09-21,1,31
2716224,2020-09-21,2,49
2716225,2020-09-22,0,1641
2716226,2020-09-22,1,14


In [17]:
#создадим колонку размера когорты из нулевого дня, чтобы высчитать процент по каждому дню
cohort_group_size = cohort_df.query('days_diff == 0')\
                                .rename(columns={'uid_count' : 'uid_cohort_size'})\
                                [['reg_date', 'uid_cohort_size']]

#присоединяем колонку размера когорты к таблице
cohort_df = cohort_df.merge(cohort_group_size, how='inner', on='reg_date')

#считаем процент вернувшихся пользователей
cohort_df['uid_retention_percent'] = cohort_df.uid_count / cohort_df.uid_cohort_size

cohort_df

,reg_date,days_diff,uid_count,uid_cohort_size,uid_retention_percent
0,1998-11-18,0,1,1,1.000000
1,1999-07-22,0,1,1,1.000000
2,1999-07-22,3,1,1,1.000000
3,1999-07-22,9,1,1,1.000000
4,1999-07-22,14,1,1,1.000000
...,...,...,...,...,...
2716223,2020-09-21,1,31,1638,0.018926
2716224,2020-09-21,2,49,1638,0.029915
2716225,2020-09-22,0,1641,1641,1.000000
2716226,2020-09-22,1,14,1641,0.008531


In [18]:
#устанваливаем период, за который планируем выгружать когорту
cohort_period = cohort_df.reg_date.max() - timedelta(days=300)

In [19]:
#преобразуем таблицу в вид сводной таблицы
final_cohort = cohort_df.query('reg_date >= @cohort_period')
final_cohort = final_cohort.assign(reg_date = final_cohort.reg_date.dt.strftime('%Y-%m-%d'))
final_cohort = final_cohort.pivot(index=['reg_date', 'uid_cohort_size'], columns = 'days_diff', values = 'uid_retention_percent')
final_cohort

,days_diff,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
reg_date,uid_cohort_size,,,,,,,,,,,,,,,,,,,,,
2019-11-28,1004,1.0,0.011952,0.040837,0.040837,0.044821,0.067729,0.064741,0.041833,0.047809,0.048805,...,0.012948,0.011952,0.011952,0.006972,0.007968,0.008964,0.020916,0.011952,0.012948,0.003984
2019-11-29,1005,1.0,0.025871,0.045771,0.056716,0.041791,0.060697,0.066667,0.058706,0.055721,0.058706,...,0.011940,0.010945,0.011940,0.011940,0.009950,0.017910,0.013930,0.008955,0.004975,NaN
2019-11-30,1008,1.0,0.023810,0.038690,0.043651,0.054563,0.072421,0.077381,0.047619,0.051587,0.053571,...,0.010913,0.014881,0.011905,0.012897,0.011905,0.010913,0.009921,0.006944,NaN,NaN
2019-12-01,1008,1.0,0.020833,0.030754,0.040675,0.044643,0.072421,0.070437,0.060516,0.050595,0.047619,...,0.014881,0.010913,0.011905,0.012897,0.012897,0.010913,0.003968,NaN,NaN,NaN
2019-12-02,1011,1.0,0.026706,0.050445,0.041543,0.050445,0.059347,0.062315,0.066271,0.050445,0.051434,...,0.010880,0.009891,0.009891,0.010880,0.015826,0.008902,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-19,1634,1.0,0.018360,0.035496,0.047124,0.034884,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-20,1636,1.0,0.024450,0.043399,0.023839,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-21,1638,1.0,0.018926,0.029915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<li>Теперь осталось запаковать действия, которые мы проделали выше, в функцию. Немного модифицируем алгоритм, чтобы можно было отфильтровать данные за определенный период.</li>

In [20]:
def get_cohort(period_from = '', period_to=''):
    #загружаем библиотеки
    import pandas as pd
    from datetime import timedelta
    
    #загружаем таблицы в переменные
    df_auth_data = pd.read_csv('auth_data.csv', sep=';')
    df_reg_data = pd.read_csv('reg_data.csv', sep=';')
            
    #преобразуем параметры в тип даты
    period_from = pd.to_datetime(period_from)
    period_to = pd.to_datetime(period_to)
    
    #Замечание по поводу применения pd.to_datetime(pd.to_datetime(...))
    #вложенный метод "to_datetime" приводит к типу object, 
    #а второй метод "to_datetime" уже приводит к типу "datetime64"
    df_auth_data['auth_ts'] = pd.to_datetime(pd.to_datetime(df_auth_data['auth_ts'], unit='s').dt.date)
    df_reg_data['reg_ts'] = pd.to_datetime(pd.to_datetime(df_reg_data['reg_ts'], unit='s').dt.date)

    #отфильтровываем необходимый период
    df_auth_data = df_auth_data.query('auth_ts >= @period_from and auth_ts <= @period_to')
    df_reg_data = df_reg_data.query('reg_ts >= @period_from and reg_ts <= @period_to')
    
    #Объединим таблицы вместе
    df_merged = df_reg_data.merge(df_auth_data, on='uid', how='left')
    
    #создадим колонку с разницей в днях между датой регистрации и датой авторизации
    df_merged['days_diff'] = (df_merged["auth_ts"] - df_merged["reg_ts"]).dt.days
    
    #сгруппируем ДФ по дате регистрации и разнице дней, чтобы сосчитать количество пользователей
    cohort_df = df_merged.groupby(['reg_ts', 'days_diff'], as_index=False)\
                        .agg({'uid' : 'nunique'})\
                        .rename(columns={'uid' : 'uid_count', 'reg_ts' : 'reg_date'})
    
    #создадим колонку размера когорты из нулевого дня, чтобы высчитать процент по каждому дню
    cohort_group_size = cohort_df.query('days_diff == 0')\
                                    .rename(columns={'uid_count' : 'uid_cohort_size'})\
                                    [['reg_date', 'uid_cohort_size']]

    #присоединяем колонку размера когорты к таблице
    cohort_df = cohort_df.merge(cohort_group_size, how='inner', on='reg_date')

    #считаем процент вернувшихся пользователей
    cohort_df['uid_retention_percent'] = cohort_df.uid_count / cohort_df.uid_cohort_size
    
    #преобразуем таблицу в вид сводной таблицы
    cohort_df = cohort_df.assign(reg_date = cohort_df.reg_date.dt.strftime('%Y-%m-%d'))
    cohort_df = cohort_df.pivot(index=['reg_date', 'uid_cohort_size'], columns = 'days_diff', values = 'uid_retention_percent')
    
    #задем стиль таблицы
    ur_style = (cohort_df
                .style
                .set_caption('ВНИМАНИЕ: ЕСЛИ ТАБЛИЦА НЕ ПОКРАСИЛАСЬ В ЦВЕТА, ТО НУЖНО ПЕРЕЧИТАТЬ НОУТБУК')  # добавляем подпись
                .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
                .highlight_null('white')  # делаем белый фон для значений NaN
                .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту

    return ur_style   

In [21]:
get_cohort(period_from = '2020-08-01', period_to='2020-08-15')

,days_diff,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
reg_date,uid_cohort_size,,,,,,,,,,,,,,,
2020-08-01,1507,100.00%,1.92%,4.45%,4.31%,5.57%,5.91%,7.63%,6.37%,4.51%,6.37%,4.78%,5.11%,5.91%,4.71%,4.51%
2020-08-02,1509,100.00%,2.58%,4.04%,4.37%,5.17%,6.49%,6.63%,6.03%,4.51%,5.30%,5.50%,5.17%,4.57%,5.50%,
2020-08-03,1512,100.00%,1.72%,4.30%,4.03%,5.56%,6.15%,7.54%,5.29%,4.96%,4.89%,5.36%,5.09%,5.62%,,
2020-08-04,1514,100.00%,2.05%,3.57%,4.69%,5.22%,5.28%,6.47%,5.15%,4.43%,4.69%,5.28%,5.61%,,,
2020-08-05,1517,100.00%,2.57%,3.49%,4.02%,5.74%,5.08%,7.78%,6.13%,4.22%,4.75%,4.75%,,,,
2020-08-06,1519,100.00%,1.71%,5.07%,5.73%,5.60%,5.73%,7.50%,6.06%,5.00%,5.79%,,,,,
2020-08-07,1522,100.00%,2.17%,3.09%,5.26%,5.32%,5.98%,7.16%,6.18%,5.12%,,,,,,
2020-08-08,1524,100.00%,2.43%,4.79%,4.20%,5.45%,5.45%,7.22%,6.30%,,,,,,,
2020-08-09,1527,100.00%,2.10%,3.93%,4.65%,5.37%,7.07%,6.29%,,,,,,,,
